# Graph Neural Networks

# PyTorch

Import:

In [6]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch
from torch_geometric.data import Data
import torch_geometric.transforms as T
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx
import torchvision.transforms as transforms
from torch_geometric.nn import GCNConv, GATConv
import torch.nn.functional as F 
from sklearn.manifold import TSNE
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import pandas as pd
import seaborn as sns
import numpy as np
import pickle
import os

In [67]:
INTERMEDIATE_PATH = 'intermediate'
OUTPUT_PATH = 'output\\ES\\SAGE'
OUTPUT_PATH_GRAPH = 'output\\ES\\Graph'
DATA_PATH = 'data'

In [8]:
historic_path = os.path.join(INTERMEDIATE_PATH, 'Input_HistoricData_ES.csv')

In [9]:
historic_path

'intermediate\\Input_HistoricData_ES.csv'

In [10]:
if os.path.exists(historic_path):
    historic_data = pd.read_csv(historic_path, sep=',')
    print("Historic data loaded successfully.")

Historic data loaded successfully.


In [11]:
historic_data = historic_data.drop('Unnamed: 0', axis=1)

In [12]:
historic_data.head(1)

,COD_PERIODO,COD_PAIS,TARGET,ALCANCE,ESTRUCTURA_VENTA,COD_SUBESTRATEGIA,DES_SUBESTRATEGIA,COD_CUC,COD_UNIDAD_NEGOCIO,DES_UNIDAD_NEGOCIO,...,PRECIO_OFERTA,FACTOR_REPETICION,REAL_VENTA_NETA_MONEDA_LOCAL,OFFER_UNIT_DSCT,OFFER_DSCT,PUP,DESLINEA,DESSUBCATEGORIA,DESESTATUS,ANIOCAMPANAINTRO
0,202217,PE,CONSULTORA,MASIVO,1430_1522_350871_350871,999,VOLUMEN,P0292090009,10,COSMETICOS,...,13.24,1.0,3624.06,0.763571,0.768688,0.002263,ESIKA,PARA OJOS,A DESCONTINUAR CONGELADO,202111.0


In [13]:
#OnehotEncoding para campaña intro:
historic_data['ES_CAMPANAINTRO'] = np.where(historic_data['COD_PERIODO']==historic_data['ANIOCAMPANAINTRO'],1,0)

In [14]:
#Eliminar columnas 
historic_data = historic_data.drop(['TARGET','COD_PAIS','ALCANCE','ANIOCAMPANAINTRO','COD_SUBESTRATEGIA',
       'DES_SUBESTRATEGIA','DESESTATUS','COD_MARCA', 'DES_MARCA'], axis=1)

In [15]:
historic_data.shape

(14970, 23)

In [16]:
historic_data.columns

Index(['COD_PERIODO', 'ESTRUCTURA_VENTA', 'COD_CUC', 'COD_UNIDAD_NEGOCIO',
       'DES_UNIDAD_NEGOCIO', 'DES_CLASE', 'COD_CATEGORIA', 'DES_CATEGORIA',
       'COD_TIPOSOLO', 'DES_TIPOSOLO', 'ES_EMPAQUE', 'ES_MUESTRA', 'DES_CUC',
       'PRECIO_NORMAL', 'PRECIO_OFERTA', 'FACTOR_REPETICION',
       'REAL_VENTA_NETA_MONEDA_LOCAL', 'OFFER_UNIT_DSCT', 'OFFER_DSCT', 'PUP',
       'DESLINEA', 'DESSUBCATEGORIA', 'ES_CAMPANAINTRO'],
      dtype='object')

In [17]:
data = historic_data.copy()

In [18]:
#Eliminar columnas 
data = data.drop(['DES_CUC','DES_UNIDAD_NEGOCIO','DES_CATEGORIA','DES_TIPOSOLO','FACTOR_REPETICION'], axis=1)

In [19]:
# Codificación de atributos categóricos 'DES_CATEGORIA' y 'DES_TIPOSOLO'
# Rellenar valores nulos con una cadena especial antes de codificar
data['COD_UNIDAD_NEGOCIO'].fillna('UNKNOWN', inplace=True)
data['DES_CLASE'].fillna('UNKNOWN', inplace=True)
data['COD_CATEGORIA'].fillna('UNKNOWN', inplace=True)
data['COD_TIPOSOLO'].fillna('UNKNOWN', inplace=True)
data['DESLINEA'].fillna('UNKNOWN', inplace=True)
data['DESSUBCATEGORIA'].fillna('UNKNOWN', inplace=True)

# Crear codificadores LabelEncoder para transformar los valores categóricos
le_unidad = LabelEncoder()
le_clase = LabelEncoder()
le_categoria = LabelEncoder()
le_tiposolo = LabelEncoder()
le_deslinea = LabelEncoder()
le_subcategoria = LabelEncoder()

# Aplicar codificación y convertir a tensores
data['COD_UNIDAD_NEGOCIO_encoded'] = le_categoria.fit_transform(data['COD_UNIDAD_NEGOCIO'])
data['DES_CLASE_encoded'] = le_tiposolo.fit_transform(data['DES_CLASE'])
data['DES_CATEGORIA_encoded'] = le_categoria.fit_transform(data['COD_CATEGORIA'])
data['DES_TIPOSOLO_encoded'] = le_tiposolo.fit_transform(data['COD_TIPOSOLO'])
data['DES_LINEA_encoded'] = le_deslinea.fit_transform(data['DESLINEA'])
data['DES_SUBCATEGORIA_encoded'] = le_subcategoria.fit_transform(data['DESSUBCATEGORIA'])

C:\Users\sheil\AppData\Local\Temp\ipykernel_28192\1551182746.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['COD_UNIDAD_NEGOCIO'].fillna('UNKNOWN', inplace=True)
C:\Users\sheil\AppData\Local\Temp\ipykernel_28192\1551182746.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy

In [20]:
data = data[['COD_PERIODO','ESTRUCTURA_VENTA','COD_CUC', 
             'ES_EMPAQUE','ES_MUESTRA','PRECIO_NORMAL','PRECIO_OFERTA',
             'REAL_VENTA_NETA_MONEDA_LOCAL', 'OFFER_UNIT_DSCT', 'OFFER_DSCT', 'PUP',
             'ES_CAMPANAINTRO',
             'COD_UNIDAD_NEGOCIO_encoded', 'DES_CLASE_encoded',
             'DES_CATEGORIA_encoded', 'DES_TIPOSOLO_encoded', 'DES_LINEA_encoded',
             'DES_SUBCATEGORIA_encoded']].copy()

In [21]:
data['ES_EMPAQUE'].fillna(0, inplace=True)
data['ES_MUESTRA'].fillna(0, inplace=True)
data['PRECIO_NORMAL'].fillna(0, inplace=True)

C:\Users\sheil\AppData\Local\Temp\ipykernel_28192\3760813815.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['ES_EMPAQUE'].fillna(0, inplace=True)
C:\Users\sheil\AppData\Local\Temp\ipykernel_28192\3760813815.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

In [22]:
data.shape

(14970, 18)

In [ ]:
import logging
# Construcción de grafos por campaña
def construir_grafos(data, output_path=OUTPUT_PATH_GRAPH):
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    grafos_por_campaña = {}
    for campaña, datos_campaña in data.groupby('COD_PERIODO'):
        logging.info(f"Generando el grafo para la campaña {campaña}")
        G = nx.Graph()

        # Agregar nodos con atributos
        for _, row in datos_campaña.iterrows():
            nodo_id = row['COD_CUC']
            atributos_nodo = {
                'ES_EMPAQUE': row['ES_EMPAQUE'],
                'ES_MUESTRA': row['ES_MUESTRA'],
                'PRECIO_NORMAL': row['PRECIO_NORMAL'],
                'DES_CATEGORIA_encoded': row['DES_CATEGORIA_encoded'],
                'DES_TIPOSOLO_encoded': row['DES_TIPOSOLO_encoded'],
                'DES_LINEA_encoded': row['DES_LINEA_encoded'],
                'DES_SUBCATEGORIA_encoded': row['DES_SUBCATEGORIA_encoded'],
                'ES_CAMPANAINTRO': row['ES_CAMPANAINTRO']
            }
            G.add_node(nodo_id, **atributos_nodo)

        # Crear aristas si están en la misma estructura de ventas
        estructura_grupos = datos_campaña.groupby('ESTRUCTURA_VENTA')
        for _, grupo in estructura_grupos:
            nodos = grupo['COD_CUC'].tolist()
            for i in range(len(nodos)):
                for j in range(i + 1, len(nodos)):
                    G.add_edge(nodos[i], nodos[j])

        # Guardar el grafo generado
        file_path = os.path.join(output_path, f"grafo_campaña_{campaña}.pkl")
        with open(file_path, "wb") as f:
            pickle.dump(G, f)
        print(f"Grafo de la campaña {campaña} guardado en {file_path}")

        grafos_por_campaña[campaña] = G

    return grafos_por_campaña

In [24]:
# Ejecutar la función con tu dataframe `data`
grafos_por_campaña = construir_grafos(data)

Grafo de la campaña 202204 guardado en output\grafo_campaña_202204.pkl
Grafo de la campaña 202205 guardado en output\grafo_campaña_202205.pkl
Grafo de la campaña 202206 guardado en output\grafo_campaña_202206.pkl
Grafo de la campaña 202207 guardado en output\grafo_campaña_202207.pkl
Grafo de la campaña 202208 guardado en output\grafo_campaña_202208.pkl
Grafo de la campaña 202209 guardado en output\grafo_campaña_202209.pkl
Grafo de la campaña 202210 guardado en output\grafo_campaña_202210.pkl
Grafo de la campaña 202211 guardado en output\grafo_campaña_202211.pkl
Grafo de la campaña 202212 guardado en output\grafo_campaña_202212.pkl
Grafo de la campaña 202213 guardado en output\grafo_campaña_202213.pkl
Grafo de la campaña 202214 guardado en output\grafo_campaña_202214.pkl
Grafo de la campaña 202215 guardado en output\grafo_campaña_202215.pkl
Grafo de la campaña 202216 guardado en output\grafo_campaña_202216.pkl
Grafo de la campaña 202217 guardado en output\grafo_campaña_202217.pkl
Grafo 

In [25]:
G = grafos_por_campaña[202404] 
plt.figure(figsize=(150, 150))
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True, node_size=20, node_color="skyblue", font_size=10, font_weight="bold", edge_color="gray")
edge_labels = nx.get_edge_attributes(G, 'tipo')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color="red")
plt.title(f"Grafo de relaciones de productos para la campaña {202404}")
plt.show()

### Modelo GraphSAGE Aplicado para la predicción de Links

In [113]:
import pandas as pd
import torch
import numpy as np
from torch_geometric.nn import SAGEConv
from sklearn.manifold import TSNE
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import networkx as nx
from torch_geometric.utils import from_networkx
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid
import random
import os
import pickle
import json
import logging

# Configuración de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class GraphSAGEModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, reg_lambda=1e-5, dropout=0.5):
        super(GraphSAGEModel, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.ln1 = torch.nn.LayerNorm(hidden_channels)  # Primera normalización
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.ln2 = torch.nn.LayerNorm(hidden_channels)  # Segunda normalización
        self.conv3 = SAGEConv(hidden_channels, out_channels)
        
        self.project_initial = torch.nn.Linear(in_channels, hidden_channels)  # Proyección para conexión residual
        self.dropout = dropout
        self.reg_lambda = reg_lambda  # Regularización L2

    def forward(self, x, edge_index):
        # Conexión inicial
        x_initial = self.project_initial(x)  # Proyectar dimensiones de entrada

        # Primera capa con LayerNorm y conexión residual
        x = F.relu(self.ln1(self.conv1(x, edge_index)))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = x + x_initial  # Conexión residual

        # Segunda capa con LayerNorm y conexión residual
        x_residual = x  # Guardar para la conexión residual
        x = F.relu(self.ln2(self.conv2(x, edge_index)))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = x + x_residual  # Conexión residual

        # Tercera capa (sin conexión residual para salida final)
        x = self.conv3(x, edge_index)
        return x

    def regularization_loss(self):
        reg_loss = 0
        for param in self.parameters():
            reg_loss += self.reg_lambda * param.norm(2)
        return reg_loss

    def entropy_loss(self, embeddings):
        """
        Calcula la pérdida de entropía para evitar oversmoothing.
        """
        prob = F.softmax(embeddings, dim=1)
        log_prob = F.log_softmax(embeddings, dim=1)
        entropy = -torch.sum(prob * log_prob) / embeddings.size(0)
        return entropy

# Modelo de predicción de enlaces
class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels):
        super(LinkPredictor, self).__init__()
        self.lin1 = torch.nn.Linear(in_channels * 2, 64)
        self.lin2 = torch.nn.Linear(64, 32)
        self.lin3 = torch.nn.Linear(32, 1)

    def forward(self, x_i, x_j):
        x = torch.cat([x_i, x_j], dim=-1)
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        return torch.sigmoid(self.lin3(x))

# Visualización de embeddings usando t-SNE
def graficar_embeddings(embeddings, campaña, output_path):
    tsne = TSNE(n_components=2, random_state=42)
    embeddings_2d = tsne.fit_transform(embeddings.cpu().numpy())
    # Guardar visualización
    plt.figure(figsize=(10, 7))
    plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], alpha=0.7, cmap='viridis')
    plt.colorbar(label="Proyección t-SNE")
    plt.title(f"Embeddings visualizados con t-SNE (Campaña {campaña})")
    plt.savefig(os.path.join(output_path, f"embeddings_campaña_{campaña}.png"))
    plt.close()

    # Guardar embeddings como tensor
    embeddings_file = os.path.join(output_path, f"embeddings_campaña_{campaña}.pt")
    torch.save(embeddings, embeddings_file)
    print(f"Embeddings guardados en {embeddings_file}")


# Generar aristas negativas
def generar_aristas_negativas(data_pg, num_negativas):
    all_nodes = torch.arange(data_pg.x.size(0))  # Todos los nodos del grafo
    existing_edges = set(map(tuple, data_pg.edge_index.t().tolist()))  # Aristas existentes
    neg_edges = set()

    while len(neg_edges) < num_negativas:
        i, j = random.sample(all_nodes.tolist(), 2)
        if (i != j) and ((i, j) not in existing_edges) and ((j, i) not in existing_edges):
            neg_edges.add((i, j))

    neg_edges = torch.tensor(list(neg_edges), dtype=torch.long).t()
    return neg_edges


In [98]:
def calculate_total_loss(model, x, edge_index, target, loss_fn, weight_entropy=1e-3):
    """
    Calcula la pérdida total combinando la pérdida estándar, la regularización L2 y la pérdida de entropía.
    
    Args:
        model: instancia del modelo GraphSAGEModel
        x: características de entrada
        edge_index: índices de las conexiones del grafo
        target: etiquetas verdaderas
        loss_fn: función de pérdida (por ejemplo, CrossEntropyLoss)
        weight_entropy: peso asignado a la pérdida de entropía

    Returns:
        Pérdida total (float)
    """
    output = model(x, edge_index)
    # Pérdida estándar
    loss_standard = loss_fn(output, target)
    # Pérdida de regularización L2
    loss_reg = model.regularization_loss()
    # Pérdida de entropía
    loss_entropy = model.entropy_loss(output)
    # Pérdida total
    total_loss = loss_standard + loss_reg + weight_entropy * loss_entropy
    return total_loss

In [ ]:
# GridSearch para buscar los mejores hiperparámetros
def buscar_mejores_hiperparametros(grafos_por_campaña, output_path=OUTPUT_PATH):
    param_grid = {
        'hidden_channels': [16, 32, 64],
        'learning_rate': [0.001, 0.0005],
        'dropout': [0.4, 0.6],
        'reg_lambda': [1e-4, 1e-5],
        'out_channels': [16, 32] 
    }

    if not os.path.exists(output_path):
        os.makedirs(output_path)

    mejores_hiperparametros = {}

    for campaña, G in grafos_por_campaña.items():
        print(f"Buscando mejores hiperparámetros para la campaña {campaña}")
        
        node_mapping = {node: idx for idx, node in enumerate(G.nodes)}
        data_pg = from_networkx(G)
        data_pg.edge_index = torch.tensor([
            [node_mapping[src], node_mapping[dst]] for src, dst in G.edges
        ]).t()

        data_pg.x = torch.tensor([
            [G.nodes[node].get('PRECIO_NORMAL', 0),
             G.nodes[node].get('DES_CATEGORIA_encoded', 0),
             G.nodes[node].get('DES_TIPOSOLO_encoded', 0),
             G.nodes[node].get('DES_LINEA_encoded', 0),
             G.nodes[node].get('DES_SUBCATEGORIA_encoded', 0),
             G.nodes[node].get('ES_EMPAQUE', 0),
             G.nodes[node].get('ES_MUESTRA', 0),
             G.nodes[node].get('ES_CAMPANAINTRO', 0)
             ] for node in G.nodes
        ], dtype=torch.float)

        num_negativas = data_pg.edge_index.size(1) * 5 
        neg_edges = generar_aristas_negativas(data_pg, num_negativas)

        all_edges = torch.cat([data_pg.edge_index, neg_edges], dim=1)
        all_labels = torch.cat([
            torch.ones(data_pg.edge_index.size(1)),
            torch.zeros(neg_edges.size(1))
        ])
        train_edges, test_edges, train_labels, test_labels = train_test_split(
            all_edges.t().numpy(), all_labels.numpy(), test_size=0.2, stratify=all_labels.numpy(), random_state=42
        )
        train_edge_index = torch.tensor(train_edges, dtype=torch.long).t()
        train_edge_attr = torch.tensor(train_labels, dtype=torch.float)
        test_edge_index = torch.tensor(test_edges, dtype=torch.long).t()
        test_edge_attr = torch.tensor(test_labels, dtype=torch.float)

        best_params = None
        best_auc = 0

        for params in ParameterGrid(param_grid):
            print(f"Probando hiperparámetros: {params}")
            model = GraphSAGEModel(
                in_channels=data_pg.x.size(1),
                hidden_channels=params['hidden_channels'],
                out_channels=params['out_channels']
            )
            model.dropout = params['dropout']
            model.reg_lambda = params['reg_lambda']

            optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
            model.train()

            for epoch in range(2000):  # Fase de búsqueda rápida
                optimizer.zero_grad()
                embeddings = model(data_pg.x, train_edge_index)
                edge_labels = train_edge_attr
                edge_predictions = torch.sigmoid(
                    (embeddings[train_edge_index[0]] * embeddings[train_edge_index[1]]).sum(dim=1)
                )
                loss = F.binary_cross_entropy(edge_predictions, edge_labels) + model.regularization_loss()
                loss.backward()
                optimizer.step()

            model.eval()
            with torch.no_grad():
                embeddings = model(data_pg.x, data_pg.edge_index)
                test_preds = torch.sigmoid(
                    (embeddings[test_edge_index[0]] * embeddings[test_edge_index[1]]).sum(dim=1)
                )
                auc_score = roc_auc_score(test_edge_attr.cpu(), test_preds.cpu())
                ###Agregar 
                test_preds_binary = (test_preds > 0.5).float()
                precision = precision_score(test_edge_attr.cpu(), test_preds_binary.cpu())                
                recall = recall_score(test_edge_attr.cpu(), test_preds_binary.cpu())
                f1 = f1_score(test_edge_attr.cpu(), test_preds_binary.cpu())
                print(f"Campaña {campaña}: AUC = {auc_score:.4f}, Precision = {precision:.4f}, Recall = {recall:.4f}, F1 = {f1:.4f}")

            if auc_score > best_auc:
                best_auc = auc_score
                best_params = params

        print(f"Los mejores hiperparametros para la campaña {campaña} son: ", best_params)
        print(f"El mejor AUC para la campaña {campaña} son: ", best_auc)
        mejores_hiperparametros[campaña] = best_params

    with open(os.path.join(output_path, "mejores_hiperparametros.json"), "w") as f:
        json.dump(mejores_hiperparametros, f, indent=4)
    print(f"Hiperparámetros guardados en {output_path}/mejores_hiperparametros.json")


# Ejecutar GridSearch
buscar_mejores_hiperparametros(grafos_por_campaña, output_path=OUTPUT_PATH)


Buscando mejores hiperparámetros para la campaña 202204
Probando hiperparámetros: {'dropout': 0.4, 'hidden_channels': 16, 'learning_rate': 0.001, 'out_channels': 16, 'reg_lambda': 0.0001}
Campaña 202204: AUC = 0.7275, Precision = 0.2154, Recall = 0.7778, F1 = 0.3373
Probando hiperparámetros: {'dropout': 0.4, 'hidden_channels': 16, 'learning_rate': 0.001, 'out_channels': 16, 'reg_lambda': 1e-05}
Campaña 202204: AUC = 0.7397, Precision = 0.2258, Recall = 0.7778, F1 = 0.3500
Probando hiperparámetros: {'dropout': 0.4, 'hidden_channels': 16, 'learning_rate': 0.001, 'out_channels': 32, 'reg_lambda': 0.0001}
Campaña 202204: AUC = 0.6940, Precision = 0.2000, Recall = 0.7037, F1 = 0.3115
Probando hiperparámetros: {'dropout': 0.4, 'hidden_channels': 16, 'learning_rate': 0.001, 'out_channels': 32, 'reg_lambda': 1e-05}
Campaña 202204: AUC = 0.6606, Precision = 0.1790, Recall = 0.7593, F1 = 0.2898
Probando hiperparámetros: {'dropout': 0.4, 'hidden_channels': 16, 'learning_rate': 0.0005, 'out_channe

In [115]:
def graficar_analisis_significatividad(importancia_permutacion, importancia_gradiente, correlaciones, campaña, output_path):
    """
    Genera gráficos para visualizar el análisis de significatividad de características con una paleta personalizada.
    Args:
        importancia_permutacion (dict): Importancia de las características por permutación.
        importancia_gradiente (tensor): Importancia de las características por gradiente.
        correlaciones (list): Correlaciones entre características y embeddings.
        campaña (int): Identificador de la campaña.
        output_path (str): Carpeta donde guardar los gráficos.
    """
    # Paleta de colores personalizada
    colores = ['#2C3E50', '#34495E', '#7F8C8D', '#95A5A6', '#4D5656', '#BDC3C7']

    # Asegurar que el directorio de salida exista
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # Gráfico de importancia por permutación
    plt.figure(figsize=(10, 6))
    plt.bar(importancia_permutacion.keys(), importancia_permutacion.values(), color=colores[0])
    plt.title(f"Importancia por Permutación - Campaña {campaña}")
    plt.ylabel("Impacto en AUC")
    plt.xlabel("Características")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(os.path.join(output_path, f"importancia_permutacion_campaña_{campaña}.png"))
    plt.close()

    # Gráfico de importancia por gradiente
    plt.figure(figsize=(10, 6))
    plt.bar(range(len(importancia_gradiente)), importancia_gradiente.numpy(), color=colores[1])
    plt.title(f"Importancia por Gradiente - Campaña {campaña}")
    plt.ylabel("Magnitud del Gradiente")
    plt.xlabel("Características")
    plt.xticks(range(len(importancia_gradiente)), [f"Feature_{i}" for i in range(len(importancia_gradiente))], rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(os.path.join(output_path, f"importancia_gradiente_campaña_{campaña}.png"))
    plt.close()

    # Gráfico de correlaciones con embeddings
    plt.figure(figsize=(10, 6))
    plt.bar(range(len(correlaciones)), correlaciones, color=colores[2])
    plt.title(f"Correlaciones con Embeddings - Campaña {campaña}")
    plt.ylabel("Coeficiente de Correlación")
    plt.xlabel("Características")
    plt.xticks(range(len(correlaciones)), [f"Feature_{i}" for i in range(len(correlaciones))], rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(os.path.join(output_path, f"correlaciones_embeddings_campaña_{campaña}.png"))
    plt.close()


In [116]:
# Evaluar modelo
def evaluar_modelo(link_predictor, test_edge_index, test_edge_attr, embeddings):
    with torch.no_grad():
        preds = link_predictor(
            embeddings[test_edge_index[0]], embeddings[test_edge_index[1]]
        ).squeeze()
        auc_score = roc_auc_score(test_edge_attr.cpu(), preds.cpu())
    return auc_score


# **1. Permutación de Features**
def calcular_importancia_permutacion(model, link_predictor, data_pg, test_edge_index, test_edge_attr, embeddings):
    baseline_auc = evaluar_modelo(link_predictor, test_edge_index, test_edge_attr, embeddings)
    importancias = {}

    for col in range(data_pg.x.size(1)):
        x_permutado = data_pg.x.clone()
        x_permutado[:, col] = x_permutado[torch.randperm(data_pg.x.size(0)), col]
        
        embeddings_permutados = model(x_permutado, data_pg.edge_index)
        auc_permutado = evaluar_modelo(link_predictor, test_edge_index, test_edge_attr, embeddings_permutados)
        importancias[f"Feature_{col}"] = baseline_auc - auc_permutado

    return importancias


# **2. Gradiente de Entrada**
def calcular_gradiente_importancia(model, data_pg):
    data_pg.x.requires_grad = True
    embeddings = model(data_pg.x, data_pg.edge_index)

    importancia_gradiente = torch.zeros(data_pg.x.size(1))
    for i in range(data_pg.x.size(0)):
        embedding = embeddings[i]
        embedding.mean().backward(retain_graph=True)
        importancia_gradiente += data_pg.x.grad[i].abs()

    return importancia_gradiente / data_pg.x.size(0)


# **3. Correlación con Embeddings**
def calcular_correlacion_embeddings(data_pg, embeddings):
    correlaciones = []
    for col in range(data_pg.x.size(1)):
        # Usar detach() para evitar problemas con gradientes
        col_data = data_pg.x[:, col].detach().cpu().numpy()
        embedding_data = embeddings.cpu().detach().numpy().mean(axis=1)
        corr = np.corrcoef(col_data, embedding_data)[0, 1]
        correlaciones.append(corr)
    return correlaciones


In [122]:
def identificar_aristas_positivas(link_predictor, test_edge_index, embeddings, threshold=0.9):
    with torch.no_grad():
        preds = link_predictor(
            embeddings[test_edge_index[0]], embeddings[test_edge_index[1]]
        ).squeeze()
        aristas_positivas = test_edge_index[:, (preds > threshold)]
    return aristas_positivas


# **2. Guardar resultados de las aristas positivas**
def guardar_aristas_positivas(aristas_positivas, campaña, output_path):
    aristas_positivas_lista = aristas_positivas.cpu().numpy().tolist()
    file_path = os.path.join(output_path, f"aristas_positivas_campaña_{campaña}.json")
    with open(file_path, "w") as f:
        json.dump(aristas_positivas_lista, f, indent=4)
    print(f"Aristas positivas guardadas en {file_path}")

def graficar_grafo_aristas_positivas(G, aristas_positivas, campaña, output_path, reverse_mapping, link_predictions=None):
    """
    Graficar el grafo original resaltando las aristas predichas como positivas.
    Args:
        G (networkx.Graph): Grafo original.
        aristas_positivas (torch.Tensor): Aristas positivas en formato de índices.
        campaña (int): Identificador de la campaña.
        output_path (str): Carpeta de salida para guardar el grafo.
        reverse_mapping (dict): Mapeo de índices de nodos a sus etiquetas originales.
        link_predictions (torch.Tensor, optional): Probabilidades predichas para las aristas.
    """
    plt.figure(figsize=(12, 10))

    # Convertir los índices de las aristas positivas a etiquetas originales
    aristas_positivas_tuples = [
        (reverse_mapping[int(u)], reverse_mapping[int(v)])
        for u, v in aristas_positivas.t().tolist()
        if reverse_mapping[int(u)] in G.nodes and reverse_mapping[int(v)] in G.nodes
    ]

    # Crear un subgrafo solo con los nodos conectados por al menos una arista positiva
    nodos_conectados = set([nodo for edge in aristas_positivas_tuples for nodo in edge])
    subgrafo = G.subgraph(nodos_conectados).copy()

    # Generar posiciones para los nodos
    pos = nx.spring_layout(subgrafo, seed=42)

    # Dibujar nodos del subgrafo
    nx.draw(subgrafo, pos, with_labels=True, node_color="lightblue", edge_color="gray", alpha=0.7, node_size=500)

    # Asignar colores a las aristas positivas
    if link_predictions is not None:
        # Normalizar las probabilidades para usarlas como gradiente de color
        probs = link_predictions.cpu().detach().numpy()
        nx.draw_networkx_edges(
            subgrafo,
            pos,
            edgelist=aristas_positivas_tuples,
            edge_color=probs,
            edge_cmap=plt.cm.Reds,  # Usar gradiente de colores
            width=2.0,
            alpha=0.8,
        )
    else:
        # Usar rojo como predeterminado si no se especifican probabilidades
        nx.draw_networkx_edges(
            subgrafo,
            pos,
            edgelist=aristas_positivas_tuples,
            edge_color="gray",
            width=2.0,
            alpha=0.8,
        )

    plt.title(f"Grafo con Aristas Positivas - Campaña {campaña}")
    plt.savefig(os.path.join(output_path, f"grafo_aristas_positivas_campaña_{campaña}.png"))
    plt.close()
    print(f"Grafo con aristas positivas guardado en {os.path.join(output_path, f'grafo_aristas_positivas_campaña_{campaña}.png')}")

In [137]:
import json
import numpy as np
import os
import matplotlib.pyplot as plt

# Función para graficar Model Loss y Model Accuracy con comparativo entre Train y Test (simplificado)
def graficar_loss_accuracy(train_losses, test_losses, train_accuracies, test_accuracies, campaña, output_path, interval=50):
    """
    Genera gráficos de líneas más simples con menos puntos de datos, promediando valores cada cierto intervalo.

    Args:
        train_losses (list): Lista de pérdidas (loss) en entrenamiento por época.
        test_losses (list): Lista de pérdidas (loss) en prueba por época.
        train_accuracies (list): Lista de precisiones (accuracy) en entrenamiento por época.
        test_accuracies (list): Lista de precisiones (accuracy) en prueba por época.
        campaña (int): Identificador de la campaña.
        output_path (str): Ruta de salida para guardar los gráficos.
        interval (int): Intervalo de promediado para reducir el número de puntos de datos graficados.
    """
    # Asegurar que el directorio de salida exista
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # Promediar los valores por intervalos
    def promedio_por_intervalo(valores, intervalo):
        return [np.mean(valores[i:i + intervalo]) for i in range(0, len(valores), intervalo)]

    # Aplicar el promedio
    train_losses_avg = promedio_por_intervalo(train_losses, interval)
    test_losses_avg = promedio_por_intervalo(test_losses, interval)
    train_accuracies_avg = promedio_por_intervalo(train_accuracies, interval)
    test_accuracies_avg = promedio_por_intervalo(test_accuracies, interval)

    epochs_avg = range(0, len(train_losses), interval)

    # Guardar los datos de métricas en un archivo JSON
    metrics_data = {
        "train_losses": train_losses,
        "test_losses": test_losses,
        "train_accuracies": train_accuracies,
        "test_accuracies": test_accuracies,
        "train_losses_avg": train_losses_avg,
        "test_losses_avg": test_losses_avg,
        "train_accuracies_avg": train_accuracies_avg,
        "test_accuracies_avg": test_accuracies_avg,
    }
    json_path = os.path.join(output_path, f"metrics_campaña_{campaña}.json")
    with open(json_path, "w") as f:
        json.dump(metrics_data, f, indent=4)
    print(f"Metrics saved to JSON: {json_path}")

    # Gráfico de pérdidas
    plt.figure(figsize=(8, 5))
    plt.plot(epochs_avg, train_losses_avg, label="Train", color="blue", linestyle="--")
    plt.plot(epochs_avg, test_losses_avg, label="Test", color="orange", linestyle="-")
    plt.title(f"Loss Comparison - Campaign {campaña}")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_path, f"loss_comparativo_simplificado_campaña_{campaña}.png"))
    plt.close()

    # Gráfico de precisión
    plt.figure(figsize=(8, 5))
    plt.plot(epochs_avg, train_accuracies_avg, label="Train", color="blue", linestyle="--")
    plt.plot(epochs_avg, test_accuracies_avg, label="Test", color="orange", linestyle="-")
    plt.title(f"Accuracy Comparison - Campaign {campaña}")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_path, f"accuracy_comparativo_simplificado_campaña_{campaña}.png"))
    plt.close()


In [145]:
def calcular_shap_values(model, data_pg, train_edge_index, campaña, output_path):
    """
    Calcula y guarda SHAP values para las características de los nodos en un modelo de Graph Neural Network.

    Args:
        model: Modelo GraphSAGE entrenado.
        data_pg: Datos del grafo (atributos de nodos y aristas).
        train_edge_index: Índices de aristas de entrenamiento.
        campaña: Identificador de la campaña.
        output_path: Ruta para guardar los valores SHAP y la gráfica.
    """
    model.eval()

    def model_predict(x):
        with torch.no_grad():
            # Ajustar las dimensiones para SHAP (procesar solo las aristas relevantes en x)
            x_tensor = torch.tensor(x, dtype=torch.float)

            # Crear embeddings basados solo en nodos presentes en `x`
            max_node_idx = x_tensor.size(0) // 2  # Dividir porque incluye nodo fuente y destino
            valid_edges = train_edge_index[:, (train_edge_index[0] < max_node_idx) & (train_edge_index[1] < max_node_idx)]
            
            # Generar embeddings para nodos seleccionados
            embeddings = model(data_pg.x[:max_node_idx], valid_edges)

            # Generar predicciones para aristas relevantes
            edge_predictions = torch.sigmoid(
                (embeddings[valid_edges[0]] * embeddings[valid_edges[1]]).sum(dim=1)
            )
            return edge_predictions.cpu().numpy()

    # Extraer atributos de nodos para aristas seleccionadas
    edge_features = torch.cat([
        data_pg.x[train_edge_index[0]],  # Nodo fuente
        data_pg.x[train_edge_index[1]],  # Nodo destino
    ], dim=1)

    # Limitar el tamaño de muestra a 100 aristas (o menos)
    num_samples = min(100, edge_features.shape[0])
    sample_data = edge_features[:num_samples].cpu().numpy()

    # Verificar dimensiones de entrada y salida
    print(f"Dimensiones de entrada (x): {sample_data.shape}")
    output = model_predict(sample_data)
    print(f"Dimensiones de salida (predicciones): {output.shape}")

    # Crear el explicador de SHAP
    explainer = shap.KernelExplainer(model_predict, sample_data)

    # Calcular SHAP values
    shap_values = explainer.shap_values(sample_data)

    # Guardar SHAP values en un archivo JSON
    shap_output_path = os.path.join(output_path, f"shap_values_campaña_{campaña}.json")
    with open(shap_output_path, "w") as f:
        json.dump(shap_values, f, indent=4)
    print(f"SHAP values guardados en: {shap_output_path}")

    # Generar y guardar el gráfico de resumen de SHAP
    plt.figure()
    shap.summary_plot(shap_values, sample_data, feature_names=[
        'Nodo Fuente - PRECIO_NORMAL', 'Nodo Fuente - DES_CATEGORIA_encoded',
        'Nodo Fuente - DES_TIPOSOLO_encoded', 'Nodo Fuente - DES_LINEA_encoded',
        'Nodo Fuente - DES_SUBCATEGORIA_encoded', 'Nodo Fuente - ES_EMPAQUE',
        'Nodo Fuente - ES_MUESTRA', 'Nodo Fuente - ES_CAMPANAINTRO',
        'Nodo Destino - PRECIO_NORMAL', 'Nodo Destino - DES_CATEGORIA_encoded',
        'Nodo Destino - DES_TIPOSOLO_encoded', 'Nodo Destino - DES_LINEA_encoded',
        'Nodo Destino - DES_SUBCATEGORIA_encoded', 'Nodo Destino - ES_EMPAQUE',
        'Nodo Destino - ES_MUESTRA', 'Nodo Destino - ES_CAMPANAINTRO',
    ], show=False)
    shap_plot_path = os.path.join(output_path, f"shap_summary_campaña_{campaña}.png")
    plt.savefig(shap_plot_path, bbox_inches="tight")
    plt.close()
    print(f"Gráfico de SHAP guardado en: {shap_plot_path}")


In [144]:
#EXPERIMENTO 1
# Entrenamiento y predicción
def entrenar_y_predecir(grafos_por_campaña, guardar_embeddings=False, output_path=OUTPUT_PATH):
    # Cargar los mejores hiperparámetros encontrados
    with open(os.path.join(output_path, "mejores_hiperparametros.json"), "r") as f:
        mejores_hiperparametros = json.load(f)

    resultados_por_campaña = {}
    paciencia = 200  # Número de épocas para aplicar early stopping
    min_delta = 1e-4  # Diferencia mínima para considerar una mejora

    for campaña, params in mejores_hiperparametros.items():
        print(f"Entrenando para la campaña {campaña} con parámetros: {params}")
        
        campaña = int(campaña)

        # Obtener el grafo de la campaña
        G = grafos_por_campaña[campaña]

        # Mapear nodos a índices
        node_mapping = {node: idx for idx, node in enumerate(G.nodes)}
        data_pg = from_networkx(G)
        data_pg.edge_index = torch.tensor([
            [node_mapping[src], node_mapping[dst]] for src, dst in G.edges
        ]).t()

        # Generar atributos de nodo
        data_pg.x = torch.tensor([
            [G.nodes[node].get('PRECIO_NORMAL', 0),
             G.nodes[node].get('DES_CATEGORIA_encoded', 0),
             G.nodes[node].get('DES_TIPOSOLO_encoded', 0),
             G.nodes[node].get('DES_LINEA_encoded', 0),
             G.nodes[node].get('DES_SUBCATEGORIA_encoded', 0),
             G.nodes[node].get('ES_EMPAQUE', 0),
             G.nodes[node].get('ES_MUESTRA', 0),
             G.nodes[node].get('ES_CAMPANAINTRO', 0)
             ] for node in G.nodes
        ], dtype=torch.float)

        # Generar aristas negativas
        num_negativas = int(data_pg.edge_index.size(1) * 5)
        neg_edges = generar_aristas_negativas(data_pg, num_negativas)

        # Crear dataset de entrenamiento y prueba
        all_edges = torch.cat([data_pg.edge_index, neg_edges], dim=1)
        all_labels = torch.cat([
            torch.ones(data_pg.edge_index.size(1)),
            torch.zeros(neg_edges.size(1))
        ])
        train_edges, test_edges, train_labels, test_labels = train_test_split(
            all_edges.t().numpy(), all_labels.numpy(), test_size=0.2, stratify=all_labels.numpy(), random_state=42
        )
        train_edge_index = torch.tensor(train_edges, dtype=torch.long).t()
        train_edge_attr = torch.tensor(train_labels, dtype=torch.float)
        test_edge_index = torch.tensor(test_edges, dtype=torch.long).t()
        test_edge_attr = torch.tensor(test_labels, dtype=torch.float)

        # Usar los mejores hiperparámetros para el modelo
        model = GraphSAGEModel(
            in_channels=data_pg.x.size(1),
            hidden_channels=params['hidden_channels'],
            out_channels=params['out_channels']
        )
        model.dropout = params['dropout']
        model.reg_lambda = params['reg_lambda']

        optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
        model.train()

        train_losses, test_losses = [], []
        train_accuracies, test_accuracies = [], []

        # Configuración de Early Stopping
        best_loss = float('inf')
        epochs_without_improvement = 0

        # Entrenar GraphSAGE
        for epoch in range(2000):  # Máximo de épocas
            optimizer.zero_grad()
            embeddings = model(data_pg.x, train_edge_index)
            edge_labels = train_edge_attr
            edge_predictions = torch.sigmoid(
                (embeddings[train_edge_index[0]] * embeddings[train_edge_index[1]]).sum(dim=1)
            )
            loss = F.binary_cross_entropy(edge_predictions, edge_labels) + model.regularization_loss()
            loss.backward()
            optimizer.step()

            # Guardar métricas de entrenamiento
            train_losses.append(loss.item())
            train_accuracy = ((edge_predictions > 0.9) == edge_labels).float().mean().item()
            train_accuracies.append(train_accuracy)

            # Validación
            with torch.no_grad():
                embeddings_test = model(data_pg.x, test_edge_index)
                test_predictions = torch.sigmoid(
                    (embeddings_test[test_edge_index[0]] * embeddings_test[test_edge_index[1]]).sum(dim=1)
                )
                test_loss = F.binary_cross_entropy(test_predictions, test_edge_attr)
                test_losses.append(test_loss.item())
                test_accuracy = ((test_predictions > 0.9) == test_edge_attr).float().mean().item()
                test_accuracies.append(test_accuracy)

            # Early Stopping: Monitorear pérdida de validación
            if test_loss < best_loss - min_delta:
                best_loss = test_loss
                epochs_without_improvement = 0
            else:
                epochs_without_improvement += 1

            # Detener entrenamiento si no hay mejora
            if epochs_without_improvement >= paciencia:
                print(f"Early stopping en la campaña {campaña}, Epoch {epoch}, Mejor pérdida de validación: {best_loss:.4f}")
                break

            if epoch % 50 == 0:
                print(f"Campaña {campaña}, Epoch {epoch}, Pérdida: {loss.item():.4f}")
        
        # Graficar pérdidas y precisión comparativas
        graficar_loss_accuracy(
            train_losses, test_losses, train_accuracies, test_accuracies, campaña, output_path
        )

        # Generar embeddings
        model.eval()
        with torch.no_grad():
            embeddings = model(data_pg.x, data_pg.edge_index)

        # Guardar y visualizar embeddings si está habilitado
        if guardar_embeddings:
            graficar_embeddings(embeddings, campaña, output_path)

        # **Análisis de SHAP values**
        print(f"--- Calculando SHAP values para la campaña {campaña} ---")
        calcular_shap_values(model, data_pg, train_edge_index, campaña, output_path)

        # Evaluación final
        link_predictor = LinkPredictor(in_channels=params['out_channels'])
        optimizer_link = torch.optim.Adam(link_predictor.parameters(), lr=params['learning_rate'])
        link_predictor.train()

        for epoch in range(2000):
            optimizer_link.zero_grad()
            preds = link_predictor(
                embeddings[test_edge_index[0]], embeddings[test_edge_index[1]]
            ).squeeze()
            loss = F.binary_cross_entropy(preds, test_edge_attr)
            loss.backward()
            optimizer_link.step()

            if epoch % 50 == 0:
                print(f"Campaña {campaña}, Epoch {epoch}, Pérdida del predictor: {loss.item():.4f}")
        
        # Evaluación de métricas
        link_predictor.eval()
        with torch.no_grad():
            test_preds = link_predictor(
                embeddings[test_edge_index[0]], embeddings[test_edge_index[1]]
            ).squeeze()

        # **Análisis de significatividad**
        print(f"--- Análisis de significatividad para campaña {campaña} ---")
        importancia_permutacion = calcular_importancia_permutacion(
            model, link_predictor, data_pg, test_edge_index, test_edge_attr, embeddings
        )
        importancia_gradiente = calcular_gradiente_importancia(model, data_pg)
        correlaciones = calcular_correlacion_embeddings(data_pg, embeddings)

        print(f"Importancia por permutación: {importancia_permutacion}")
        print(f"Importancia por gradiente: {importancia_gradiente}")
        print(f"Correlaciones con embeddings: {correlaciones}")

        graficar_analisis_significatividad(
            importancia_permutacion,
            importancia_gradiente,
            correlaciones,
            campaña,
            output_path
        )

        # Reverse Mapping
        reverse_mapping = {idx: node for node, idx in node_mapping.items()}

        # Identificar, guardar y graficar aristas positivas
        aristas_positivas = identificar_aristas_positivas(link_predictor, test_edge_index, embeddings)
        guardar_aristas_positivas(aristas_positivas, campaña, output_path)
        graficar_grafo_aristas_positivas(G, aristas_positivas, campaña, output_path, reverse_mapping)
    
    # Guardar resultados
    with open(os.path.join(output_path, "resultados_por_campaña.json"), "w") as f:
        json.dump(resultados_por_campaña, f, indent=4)

    print(f"Resultados guardados en {os.path.join(output_path, 'resultados_por_campaña.json')}")

# Entrenar y predecir usando los mejores hiperparámetros
entrenar_y_predecir(grafos_por_campaña, guardar_embeddings=True)

Entrenando para la campaña 202204 con parámetros: {'dropout': 0.6, 'hidden_channels': 16, 'learning_rate': 0.0005, 'out_channels': 16, 'reg_lambda': 0.0001}
Campaña 202204, Epoch 0, Pérdida: 83.3489
Campaña 202204, Epoch 50, Pérdida: 83.3487
Campaña 202204, Epoch 100, Pérdida: 83.3484
Campaña 202204, Epoch 150, Pérdida: 83.3482
Early stopping en la campaña 202204, Epoch 200, Mejor pérdida de validación: 82.6625
Metrics saved to JSON: output\ES\SAGE\metrics_campaña_202204.json


C:\Users\sheil\AppData\Local\Temp\ipykernel_28192\1666358028.py:89: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], alpha=0.7, cmap='viridis')


Embeddings guardados en output\ES\SAGE\embeddings_campaña_202204.pt
--- Calculando SHAP values para la campaña 202204 ---
Dimensiones de entrada (x): (100, 16)
Dimensiones de salida (predicciones): (1291,)


ValueError: operands could not be broadcast together with shapes (1291,) (100,) 

In [ ]:
#EXPERIMENTO 1
# Entrenamiento y predicción
def entrenar_y_predecir(grafos_por_campaña, guardar_embeddings=False, output_path=OUTPUT_PATH):
    # Cargar los mejores hiperparámetros encontrados
    with open(os.path.join(output_path, "mejores_hiperparametros.json"), "r") as f:
        mejores_hiperparametros = json.load(f)

    resultados_por_campaña = {}
    paciencia = 200  # Número de épocas para aplicar early stopping
    min_delta = 1e-4  # Diferencia mínima para considerar una mejora

    for campaña, params in mejores_hiperparametros.items():
        print(f"Entrenando para la campaña {campaña} con parámetros: {params}")
        
        campaña = int(campaña)

        # Obtener el grafo de la campaña
        G = grafos_por_campaña[campaña]

        # Mapear nodos a índices
        node_mapping = {node: idx for idx, node in enumerate(G.nodes)}
        data_pg = from_networkx(G)
        data_pg.edge_index = torch.tensor([
            [node_mapping[src], node_mapping[dst]] for src, dst in G.edges
        ]).t()

        # Generar atributos de nodo
        data_pg.x = torch.tensor([
            [G.nodes[node].get('PRECIO_NORMAL', 0),
             G.nodes[node].get('DES_CATEGORIA_encoded', 0),
             G.nodes[node].get('DES_TIPOSOLO_encoded', 0),
             G.nodes[node].get('DES_LINEA_encoded', 0),
             G.nodes[node].get('DES_SUBCATEGORIA_encoded', 0),
             G.nodes[node].get('ES_EMPAQUE', 0),
             G.nodes[node].get('ES_MUESTRA', 0),
             G.nodes[node].get('ES_CAMPANAINTRO', 0)
             ] for node in G.nodes
        ], dtype=torch.float)

        # Generar aristas negativas
        num_negativas = int(data_pg.edge_index.size(1) * 5)
        neg_edges = generar_aristas_negativas(data_pg, num_negativas)

        # Crear dataset de entrenamiento y prueba
        all_edges = torch.cat([data_pg.edge_index, neg_edges], dim=1)
        all_labels = torch.cat([
            torch.ones(data_pg.edge_index.size(1)),
            torch.zeros(neg_edges.size(1))
        ])
        train_edges, test_edges, train_labels, test_labels = train_test_split(
            all_edges.t().numpy(), all_labels.numpy(), test_size=0.2, stratify=all_labels.numpy(), random_state=42
        )
        train_edge_index = torch.tensor(train_edges, dtype=torch.long).t()
        train_edge_attr = torch.tensor(train_labels, dtype=torch.float)
        test_edge_index = torch.tensor(test_edges, dtype=torch.long).t()
        test_edge_attr = torch.tensor(test_labels, dtype=torch.float)

        # Usar los mejores hiperparámetros para el modelo
        model = GraphSAGEModel(
            in_channels=data_pg.x.size(1),
            hidden_channels=params['hidden_channels'],
            out_channels=params['out_channels']
        )
        model.dropout = params['dropout']
        model.reg_lambda = params['reg_lambda']

        optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
        model.train()

        train_losses, test_losses = [], []
        train_accuracies, test_accuracies = [], []

        # Configuración de Early Stopping
        best_loss = float('inf')
        epochs_without_improvement = 0

        # Entrenar GraphSAGE
        for epoch in range(2000):  # Máximo de épocas
            optimizer.zero_grad()
            embeddings = model(data_pg.x, train_edge_index)
            edge_labels = train_edge_attr
            edge_predictions = torch.sigmoid(
                (embeddings[train_edge_index[0]] * embeddings[train_edge_index[1]]).sum(dim=1)
            )
            loss = F.binary_cross_entropy(edge_predictions, edge_labels) + model.regularization_loss()
            loss.backward()
            optimizer.step()

            # Guardar métricas de entrenamiento
            train_losses.append(loss.item())
            train_accuracy = ((edge_predictions > 0.5) == edge_labels).float().mean().item()
            train_accuracies.append(train_accuracy)

            # Validación
            with torch.no_grad():
                embeddings_test = model(data_pg.x, test_edge_index)
                test_predictions = torch.sigmoid(
                    (embeddings_test[test_edge_index[0]] * embeddings_test[test_edge_index[1]]).sum(dim=1)
                )
                test_loss = F.binary_cross_entropy(test_predictions, test_edge_attr)
                test_losses.append(test_loss.item())
                test_accuracy = ((test_predictions > 0.5) == test_edge_attr).float().mean().item()
                test_accuracies.append(test_accuracy)

            # Early Stopping: Monitorear pérdida de validación
            if test_loss < best_loss - min_delta:
                best_loss = test_loss
                epochs_without_improvement = 0
            else:
                epochs_without_improvement += 1

            # Detener entrenamiento si no hay mejora
            if epochs_without_improvement >= paciencia:
                print(f"Early stopping en la campaña {campaña}, Epoch {epoch}, Mejor pérdida de validación: {best_loss:.4f}")
                break

            if epoch % 50 == 0:
                print(f"Campaña {campaña}, Epoch {epoch}, Pérdida: {loss.item():.4f}")

        # Graficar pérdidas y precisión comparativas
        graficar_loss_accuracy(
            train_losses, test_losses, train_accuracies, test_accuracies, campaña, output_path
        )

        # Generar embeddings
        model.eval()
        with torch.no_grad():
            embeddings = model(data_pg.x, data_pg.edge_index)

        # Guardar y visualizar embeddings si está habilitado
        if guardar_embeddings:
            graficar_embeddings(embeddings, campaña, output_path)

        # Evaluación final
        link_predictor = LinkPredictor(in_channels=params['out_channels'])
        optimizer_link = torch.optim.Adam(link_predictor.parameters(), lr=params['learning_rate'])
        link_predictor.train()

        for epoch in range(2000):
            optimizer_link.zero_grad()
            preds = link_predictor(
                embeddings[test_edge_index[0]], embeddings[test_edge_index[1]]
            ).squeeze()
            loss = F.binary_cross_entropy(preds, test_edge_attr)
            loss.backward()
            optimizer_link.step()

            if epoch % 50 == 0:
                print(f"Campaña {campaña}, Epoch {epoch}, Pérdida del predictor: {loss.item():.4f}")

    print(f"Resultados guardados en {os.path.join(output_path, 'resultados_por_campaña.json')}")

# Entrenar y predecir usando los mejores hiperparámetros
entrenar_y_predecir(grafos_por_campaña, guardar_embeddings=True)

#### Modelo GCN para la predicción de Links

In [57]:
OUTPUT_PATH = 'output\\ES\\GCN'

In [55]:
# Modelo GCN con regularización L2
from torch_geometric.nn import GCNConv

class GCNModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.dropout = 0.5
        self.reg_lambda = 1e-5  # Regularización L2

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv3(x, edge_index)
        return x

    def regularization_loss(self):
        reg_loss = 0
        for param in self.parameters():
            reg_loss += self.reg_lambda * param.norm(2)
        return reg_loss

In [64]:
# GridSearch para buscar los mejores hiperparámetros (adaptado para GCN)
def buscar_mejores_hiperparametros_gcn(grafos_por_campaña, output_path="output"):
    param_grid = {
        'hidden_channels': [32, 64],
        'learning_rate': [0.001, 0.0001],
        'dropout': [0.3, 0.5],
        'reg_lambda': [1e-5, 1e-6]
    }

    if not os.path.exists(output_path):
        os.makedirs(output_path)

    mejores_hiperparametros = {}

    for campaña, G in grafos_por_campaña.items():
        print(f"Buscando mejores hiperparámetros para la campaña {campaña}")
        
        node_mapping = {node: idx for idx, node in enumerate(G.nodes)}
        data_pg = from_networkx(G)
        data_pg.edge_index = torch.tensor([
            [node_mapping[src], node_mapping[dst]] for src, dst in G.edges
        ]).t()

        data_pg.x = torch.tensor([
            [G.nodes[node].get('PRECIO_NORMAL', 0),
             G.nodes[node].get('DES_CATEGORIA_encoded', 0),
             G.nodes[node].get('DES_TIPOSOLO_encoded', 0),
             G.nodes[node].get('DES_LINEA_encoded', 0),
             G.nodes[node].get('DES_SUBCATEGORIA_encoded', 0),
             G.nodes[node].get('ES_EMPAQUE', 0),
             G.nodes[node].get('ES_MUESTRA', 0),
             G.nodes[node].get('ES_CAMPANAINTRO', 0)
             ] for node in G.nodes
        ], dtype=torch.float)

        num_negativas = data_pg.edge_index.size(1)
        neg_edges = generar_aristas_negativas(data_pg, num_negativas)

        all_edges = torch.cat([data_pg.edge_index, neg_edges], dim=1)
        all_labels = torch.cat([
            torch.ones(data_pg.edge_index.size(1)),
            torch.zeros(neg_edges.size(1))
        ])
        train_edges, test_edges, train_labels, test_labels = train_test_split(
            all_edges.t().numpy(), all_labels.numpy(), test_size=0.2, stratify=all_labels.numpy(), random_state=42
        )
        train_edge_index = torch.tensor(train_edges, dtype=torch.long).t()
        train_edge_attr = torch.tensor(train_labels, dtype=torch.float)
        test_edge_index = torch.tensor(test_edges, dtype=torch.long).t()
        test_edge_attr = torch.tensor(test_labels, dtype=torch.float)

        best_params = None
        best_auc = 0

        for params in ParameterGrid(param_grid):
            print(f"Probando hiperparámetros: {params}")
            model = GCNModel(
                in_channels=data_pg.x.size(1),
                hidden_channels=params['hidden_channels'],
                out_channels=16
            )
            model.dropout = params['dropout']
            model.reg_lambda = params['reg_lambda']

            optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
            model.train()

            for epoch in range(2000):  # Fase de búsqueda rápida
                optimizer.zero_grad()
                embeddings = model(data_pg.x, train_edge_index)
                edge_labels = train_edge_attr
                edge_predictions = torch.sigmoid(
                    (embeddings[train_edge_index[0]] * embeddings[train_edge_index[1]]).sum(dim=1)
                )
                loss = F.binary_cross_entropy(edge_predictions, edge_labels) + model.regularization_loss()
                loss.backward()
                optimizer.step()

            model.eval()
            with torch.no_grad():
                embeddings = model(data_pg.x, data_pg.edge_index)
                test_preds = torch.sigmoid(
                    (embeddings[test_edge_index[0]] * embeddings[test_edge_index[1]]).sum(dim=1)
                )
                auc_score = roc_auc_score(test_edge_attr.cpu(), test_preds.cpu())

            if auc_score > best_auc:
                best_auc = auc_score
                best_params = params

        mejores_hiperparametros[campaña] = best_params

    with open(os.path.join(output_path, "mejores_hiperparametros.json"), "w") as f:
        json.dump(mejores_hiperparametros, f, indent=4)
    print(f"Hiperparámetros guardados en {output_path}/mejores_hiperparametros.json")

In [65]:
# Ejecutar GridSearch
buscar_mejores_hiperparametros(grafos_por_campaña, output_path=OUTPUT_PATH)

Buscando mejores hiperparámetros para la campaña 202204
Probando hiperparámetros: {'dropout': 0.3, 'hidden_channels': 32, 'learning_rate': 0.001, 'reg_lambda': 1e-05}
Probando hiperparámetros: {'dropout': 0.3, 'hidden_channels': 32, 'learning_rate': 0.001, 'reg_lambda': 1e-06}
Probando hiperparámetros: {'dropout': 0.3, 'hidden_channels': 32, 'learning_rate': 0.0001, 'reg_lambda': 1e-05}
Probando hiperparámetros: {'dropout': 0.3, 'hidden_channels': 32, 'learning_rate': 0.0001, 'reg_lambda': 1e-06}
Probando hiperparámetros: {'dropout': 0.3, 'hidden_channels': 64, 'learning_rate': 0.001, 'reg_lambda': 1e-05}
Probando hiperparámetros: {'dropout': 0.3, 'hidden_channels': 64, 'learning_rate': 0.001, 'reg_lambda': 1e-06}
Probando hiperparámetros: {'dropout': 0.3, 'hidden_channels': 64, 'learning_rate': 0.0001, 'reg_lambda': 1e-05}
Probando hiperparámetros: {'dropout': 0.3, 'hidden_channels': 64, 'learning_rate': 0.0001, 'reg_lambda': 1e-06}
Probando hiperparámetros: {'dropout': 0.5, 'hidden_c

In [66]:
# Entrenamiento y predicción
def entrenar_y_predecir(grafos_por_campaña, guardar_embeddings=False, output_path=OUTPUT_PATH):
    # Cargar los mejores hiperparámetros encontrados
    with open(os.path.join(output_path, "mejores_hiperparametros.json"), "r") as f:
        mejores_hiperparametros = json.load(f)

    resultados_por_campaña = {}

    for campaña, params in mejores_hiperparametros.items():
        print(f"Entrenando para la campaña {campaña} con parámetros: {params}")
        
        campaña = int(campaña)

        # Obtener el grafo de la campaña
        G = grafos_por_campaña[campaña]

        # Mapear nodos a índices
        node_mapping = {node: idx for idx, node in enumerate(G.nodes)}
        data_pg = from_networkx(G)
        data_pg.edge_index = torch.tensor([
            [node_mapping[src], node_mapping[dst]] for src, dst in G.edges
        ]).t()

        # Generar atributos de nodo
        data_pg.x = torch.tensor([
            [G.nodes[node].get('PRECIO_NORMAL', 0),
             G.nodes[node].get('DES_CATEGORIA_encoded', 0),
             G.nodes[node].get('DES_TIPOSOLO_encoded', 0),
             G.nodes[node].get('DES_LINEA_encoded', 0),
             G.nodes[node].get('DES_SUBCATEGORIA_encoded', 0),
             G.nodes[node].get('ES_EMPAQUE', 0),
             G.nodes[node].get('ES_MUESTRA', 0),
             G.nodes[node].get('ES_CAMPANAINTRO', 0)
             ] for node in G.nodes
        ], dtype=torch.float)

        # Generar aristas negativas
        num_negativas = data_pg.edge_index.size(1)
        neg_edges = generar_aristas_negativas(data_pg, num_negativas)

        # Crear dataset de entrenamiento y prueba
        all_edges = torch.cat([data_pg.edge_index, neg_edges], dim=1)
        all_labels = torch.cat([
            torch.ones(data_pg.edge_index.size(1)),
            torch.zeros(neg_edges.size(1))
        ])
        train_edges, test_edges, train_labels, test_labels = train_test_split(
            all_edges.t().numpy(), all_labels.numpy(), test_size=0.2, stratify=all_labels.numpy(), random_state=42
        )
        train_edge_index = torch.tensor(train_edges, dtype=torch.long).t()
        train_edge_attr = torch.tensor(train_labels, dtype=torch.float)
        test_edge_index = torch.tensor(test_edges, dtype=torch.long).t()
        test_edge_attr = torch.tensor(test_labels, dtype=torch.float)

        # Usar los mejores hiperparámetros para el modelo
        model = GCNModel(
            in_channels=data_pg.x.size(1),
            hidden_channels=params['hidden_channels'],
            out_channels=16
        )
        model.dropout = params['dropout']
        model.reg_lambda = params['reg_lambda']

        optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
        model.train()

        train_losses, test_losses = [], []
        train_accuracies, test_accuracies = [], []

        # Entrenar GCN
        for epoch in range(2000):  # Entrenamiento completo
            optimizer.zero_grad()
            embeddings = model(data_pg.x, train_edge_index)
            edge_labels = train_edge_attr
            edge_predictions = torch.sigmoid(
                (embeddings[train_edge_index[0]] * embeddings[train_edge_index[1]]).sum(dim=1)
            )
            loss = F.binary_cross_entropy(edge_predictions, edge_labels) + model.regularization_loss()
            loss.backward()
            optimizer.step()

            # Guardar métricas de entrenamiento
            train_losses.append(loss.item())
            train_accuracy = ((edge_predictions > 0.5) == edge_labels).float().mean().item()
            train_accuracies.append(train_accuracy)

            # Test Step
            with torch.no_grad():
                embeddings_test = model(data_pg.x, test_edge_index)
                test_predictions = torch.sigmoid(
                    (embeddings_test[test_edge_index[0]] * embeddings_test[test_edge_index[1]]).sum(dim=1)
                )
                test_loss = F.binary_cross_entropy(test_predictions, test_edge_attr)
                test_losses.append(test_loss.item())
                test_accuracy = ((test_predictions > 0.5) == test_edge_attr).float().mean().item()
                test_accuracies.append(test_accuracy)

            if epoch % 50 == 0:
                print(f"Campaña {campaña}, Epoch {epoch}, Pérdida: {loss.item():.4f}")

        # Graficar pérdidas y precisión comparativas
        graficar_loss_accuracy(
            train_losses, test_losses, train_accuracies, test_accuracies, campaña, output_path
        )

        # Generar embeddings
        model.eval()
        with torch.no_grad():
            embeddings = model(data_pg.x, data_pg.edge_index)

        # Guardar y visualizar embeddings si está habilitado
        if guardar_embeddings:
            graficar_embeddings(embeddings, campaña, output_path)

        # Evaluación final
        link_predictor = LinkPredictor(in_channels=16)
        optimizer_link = torch.optim.Adam(link_predictor.parameters(), lr=params['learning_rate'])
        link_predictor.train()

        for epoch in range(2000):
            optimizer_link.zero_grad()
            preds = link_predictor(
                embeddings[test_edge_index[0]], embeddings[test_edge_index[1]]
            ).squeeze()
            loss = F.binary_cross_entropy(preds, test_edge_attr)
            loss.backward()
            optimizer_link.step()

            if epoch % 50 == 0:
                print(f"Campaña {campaña}, Epoch {epoch}, Pérdida del predictor: {loss.item():.4f}")

        # Evaluación de métricas
        link_predictor.eval()
        with torch.no_grad():
            test_preds = link_predictor(
                embeddings[test_edge_index[0]], embeddings[test_edge_index[1]]
            ).squeeze()
            auc_score = roc_auc_score(test_edge_attr.cpu(), test_preds.cpu())
            test_preds_binary = (test_preds > 0.5).float()
            precision = precision_score(test_edge_attr.cpu(), test_preds_binary.cpu())
            recall = recall_score(test_edge_attr.cpu(), test_preds_binary.cpu())
            f1 = f1_score(test_edge_attr.cpu(), test_preds_binary.cpu())

            print(f"Campaña {campaña}: AUC = {auc_score:.4f}, Precision = {precision:.4f}, Recall = {recall:.4f}, F1 = {f1:.4f}")
            resultados_por_campaña[campaña] = {
                "AUC": auc_score,
                "Precision": precision,
                "Recall": recall,
                "F1": f1
            }

        # **Análisis de significatividad**
        print(f"--- Análisis de significatividad para campaña {campaña} ---")
        importancia_permutacion = calcular_importancia_permutacion(
            model, link_predictor, data_pg, test_edge_index, test_edge_attr, embeddings
        )
        importancia_gradiente = calcular_gradiente_importancia(model, data_pg)
        correlaciones = calcular_correlacion_embeddings(data_pg, embeddings)

        print(f"Importancia por permutación: {importancia_permutacion}")
        print(f"Importancia por gradiente: {importancia_gradiente}")
        print(f"Correlaciones con embeddings: {correlaciones}")

        graficar_analisis_significatividad(
            importancia_permutacion,
            importancia_gradiente,
            correlaciones,
            campaña,
            output_path
        )

        # Reverse Mapping
        reverse_mapping = {idx: node for node, idx in node_mapping.items()}

        # Identificar, guardar y graficar aristas positivas
        aristas_positivas = identificar_aristas_positivas(link_predictor, test_edge_index, embeddings)
        guardar_aristas_positivas(aristas_positivas, campaña, output_path)
        graficar_grafo_aristas_positivas(G, aristas_positivas, campaña, output_path, reverse_mapping)

    # Guardar resultados
    with open(os.path.join(output_path, "resultados_por_campaña.json"), "w") as f:
        json.dump(resultados_por_campaña, f, indent=4)

    print(f"Resultados guardados en {os.path.join(output_path, 'resultados_por_campaña.json')}")

# Entrenar y predecir usando los mejores hiperparámetros
entrenar_y_predecir(grafos_por_campaña, guardar_embeddings=True)


Entrenando para la campaña 202204 con parámetros: {'dropout': 0.3, 'hidden_channels': 32, 'learning_rate': 0.001, 'reg_lambda': 1e-06}
Campaña 202204, Epoch 0, Pérdida: 50.0000
Campaña 202204, Epoch 50, Pérdida: 50.0000
Campaña 202204, Epoch 100, Pérdida: 50.0000
Campaña 202204, Epoch 150, Pérdida: 14.9245
Campaña 202204, Epoch 200, Pérdida: 1.0066
Campaña 202204, Epoch 250, Pérdida: 0.8807
Campaña 202204, Epoch 300, Pérdida: 0.6979
Campaña 202204, Epoch 350, Pérdida: 0.7044
Campaña 202204, Epoch 400, Pérdida: 0.6704
Campaña 202204, Epoch 450, Pérdida: 0.6827
Campaña 202204, Epoch 500, Pérdida: 0.6297
Campaña 202204, Epoch 550, Pérdida: 0.6378
Campaña 202204, Epoch 600, Pérdida: 0.6640
Campaña 202204, Epoch 650, Pérdida: 0.6490
Campaña 202204, Epoch 700, Pérdida: 0.6235
Campaña 202204, Epoch 750, Pérdida: 0.6323
Campaña 202204, Epoch 800, Pérdida: 0.6274
Campaña 202204, Epoch 850, Pérdida: 0.6355
Campaña 202204, Epoch 900, Pérdida: 0.6186
Campaña 202204, Epoch 950, Pérdida: 0.6280
Camp

C:\Users\sheil\AppData\Local\Temp\ipykernel_28192\2261630590.py:68: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], alpha=0.7, cmap='viridis')


Embeddings guardados en output\ES\GCN\embeddings_campaña_202204.pt
Campaña 202204, Epoch 0, Pérdida del predictor: 0.7039
Campaña 202204, Epoch 50, Pérdida del predictor: 0.4397
Campaña 202204, Epoch 100, Pérdida del predictor: 0.2523
Campaña 202204, Epoch 150, Pérdida del predictor: 0.1374
Campaña 202204, Epoch 200, Pérdida del predictor: 0.0719
Campaña 202204, Epoch 250, Pérdida del predictor: 0.0415
Campaña 202204, Epoch 300, Pérdida del predictor: 0.0250
Campaña 202204, Epoch 350, Pérdida del predictor: 0.0156
Campaña 202204, Epoch 400, Pérdida del predictor: 0.0103
Campaña 202204, Epoch 450, Pérdida del predictor: 0.0071
Campaña 202204, Epoch 500, Pérdida del predictor: 0.0052
Campaña 202204, Epoch 550, Pérdida del predictor: 0.0039
Campaña 202204, Epoch 600, Pérdida del predictor: 0.0030
Campaña 202204, Epoch 650, Pérdida del predictor: 0.0024
Campaña 202204, Epoch 700, Pérdida del predictor: 0.0019
Campaña 202204, Epoch 750, Pérdida del predictor: 0.0016
Campaña 202204, Epoch 80

c:\Users\sheil\OneDrive\Desktop\Proyecto_Tesis\tesis\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\sheil\OneDrive\Desktop\Proyecto_Tesis\tesis\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


Importancia por permutación: {'Feature_0': 0.20387517146776402, 'Feature_1': 0.0, 'Feature_2': 0.3443072702331962, 'Feature_3': 0.5147462277091907, 'Feature_4': 0.1826131687242798, 'Feature_5': 0.0, 'Feature_6': 0.0, 'Feature_7': 0.0}
Importancia por gradiente: tensor([1.2032e-03, 2.5328e-03, 2.7161e-03, 1.9835e-03, 2.5640e-03, 5.1757e-13,
        2.1555e-13, 8.7086e-14])
Correlaciones con embeddings: [0.182132081275065, 0.02204743380266657, -0.28723330555162235, 0.014361346228463545, -0.06765362322912437, nan, nan, nan]
Aristas positivas guardadas en output\ES\GCN\aristas_positivas_campaña_202204.json
Grafo con aristas positivas guardado en output\ES\GCN\grafo_aristas_positivas_campaña_202204.png
Entrenando para la campaña 202205 con parámetros: {'dropout': 0.3, 'hidden_channels': 64, 'learning_rate': 0.001, 'reg_lambda': 1e-05}
Campaña 202205, Epoch 0, Pérdida: 50.0002
Campaña 202205, Epoch 50, Pérdida: 50.0001
Campaña 202205, Epoch 100, Pérdida: 1.4266
Campaña 202205, Epoch 150, Pér

...

Importancia por permutación: {'Feature_0': 0.12811713566689553, 'Feature_1': 0.019732326698695957, 'Feature_2': 0.274021962937543, 'Feature_3': 0.18960192175703505, 'Feature_4': 0.07990162434225578, 'Feature_5': -0.0002287805994050096, 'Feature_6': 0.0, 'Feature_7': 0.0}
Importancia por gradiente: tensor([7.0369e-04, 1.1853e-03, 8.4563e-04, 6.9954e-04, 1.1571e-03, 1.4129e-03,
        1.7343e-10, 6.7263e-03])
Correlaciones con embeddings: [-0.5228453686133228, -0.10751371813958806, -0.012384711437122253, -0.00250055924007937, -0.007905905765142274, 0.12693285077794528, nan, 0.06176429008441501]
Aristas positivas guardadas en output\ES\GCN\aristas_positivas_campaña_202403.json
Grafo con aristas positivas guardado en output\ES\GCN\grafo_aristas_positivas_campaña_202403.png
Entrenando para la campaña 202404 con parámetros: {'dropout': 0.3, 'hidden_channels': 64, 'learning_rate': 0.0001, 'reg_lambda': 1e-06}
Campaña 202404, Epoch 0, Pérdida: 49.9210
Campaña 202404, Epoch 50, Pérdida: 49.921

C:\Users\sheil\AppData\Local\Temp\ipykernel_28192\2261630590.py:68: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], alpha=0.7, cmap='viridis')


Embeddings guardados en output\ES\GCN\embeddings_campaña_202404.pt
Campaña 202404, Epoch 0, Pérdida del predictor: 0.6961
Campaña 202404, Epoch 50, Pérdida del predictor: 0.6588
Campaña 202404, Epoch 100, Pérdida del predictor: 0.6279
Campaña 202404, Epoch 150, Pérdida del predictor: 0.5949
Campaña 202404, Epoch 200, Pérdida del predictor: 0.5590
Campaña 202404, Epoch 250, Pérdida del predictor: 0.5208
Campaña 202404, Epoch 300, Pérdida del predictor: 0.4814
Campaña 202404, Epoch 350, Pérdida del predictor: 0.4436
Campaña 202404, Epoch 400, Pérdida del predictor: 0.4078
Campaña 202404, Epoch 450, Pérdida del predictor: 0.3760
Campaña 202404, Epoch 500, Pérdida del predictor: 0.3483
Campaña 202404, Epoch 550, Pérdida del predictor: 0.3268
Campaña 202404, Epoch 600, Pérdida del predictor: 0.3100
Campaña 202404, Epoch 650, Pérdida del predictor: 0.2967
Campaña 202404, Epoch 700, Pérdida del predictor: 0.2854
Campaña 202404, Epoch 750, Pérdida del predictor: 0.2754
Campaña 202404, Epoch 80

c:\Users\sheil\OneDrive\Desktop\Proyecto_Tesis\tesis\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\sheil\OneDrive\Desktop\Proyecto_Tesis\tesis\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


Importancia por permutación: {'Feature_0': 0.37278481012658227, 'Feature_1': 0.0066455696202532, 'Feature_2': 0.07689873417721516, 'Feature_3': 0.017721518987341645, 'Feature_4': -0.00031645569620253333, 'Feature_5': 0.0, 'Feature_6': 0.0, 'Feature_7': 0.0}
Importancia por gradiente: tensor([0.0002, 0.0009, 0.0005, 0.0003, 0.0005, 0.0005, 0.0001, 0.0007])
Correlaciones con embeddings: [-0.5263031366915439, -0.3863002084123749, 0.21111132696246554, -0.373990654733287, -0.6304247605695736, -0.08646947111909832, nan, -0.0399887259759926]
Aristas positivas guardadas en output\ES\GCN\aristas_positivas_campaña_202404.json
Grafo con aristas positivas guardado en output\ES\GCN\grafo_aristas_positivas_campaña_202404.png
Resultados guardados en output\ES\GCN\resultados_por_campaña.json


##### Comparar Modelos

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

def graficar_curva_roc(test_edge_attr, preds, atributos_1, auc_1, atributos_2, auc_2, output_path="output", campaña="General"):
    """
    Genera y guarda la curva ROC comparando dos configuraciones de atributos.

    Args:
        test_edge_attr (array): Valores reales de las etiquetas (0 o 1).
        preds (array): Predicciones para cada configuración.
        atributos_1 (str): Descripción de los atributos del primer modelo.
        auc_1 (float): Valor AUC del primer modelo.
        atributos_2 (str): Descripción de los atributos del segundo modelo.
        auc_2 (float): Valor AUC del segundo modelo.
        output_path (str): Directorio donde guardar la gráfica.
        campaña (str): Identificador de la campaña.
    """
    # Calcular las curvas ROC para ambas configuraciones
    fpr_1, tpr_1, _ = roc_curve(test_edge_attr, preds[0])
    fpr_2, tpr_2, _ = roc_curve(test_edge_attr, preds[1])

    # Graficar las curvas ROC
    plt.figure(figsize=(8, 6))
    plt.plot(fpr_1, tpr_1, label=f"AUC ({atributos_1}) = {auc_1:.2f}", color="blue", linestyle="-")
    plt.plot(fpr_2, tpr_2, label=f"AUC ({atributos_2}) = {auc_2:.2f}", color="orange", linestyle="-")
    plt.plot([0, 1], [0, 1], color="black", linestyle="--")  # Línea diagonal de referencia

    # Configurar el gráfico
    plt.title("Receiver operating characteristic")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.tight_layout()

    # Guardar la figura
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    plt.savefig(os.path.join(output_path, f"curva_roc_campaña_{campaña}.png"))
    plt.close()
    print(f"Curva ROC guardada en {output_path}/curva_roc_campaña_{campaña}.png")

# Ejemplo de uso
# Simula valores de etiquetas reales y predicciones para dos modelos
import numpy as np
test_edge_attr = np.random.randint(0, 2, 1000)  # Etiquetas reales
preds_1 = np.random.rand(1000)  # Predicciones modelo 1
preds_2 = np.random.rand(1000)  # Predicciones modelo 2

# Calcular AUC
auc_1 = auc(roc_curve(test_edge_attr, preds_1)[0], roc_curve(test_edge_attr, preds_1)[1])
auc_2 = auc(roc_curve(test_edge_attr, preds_2)[0], roc_curve(test_edge_attr, preds_2)[1])

graficar_curva_roc(
    test_edge_attr,
    preds=[preds_1, preds_2],
    atributos_1="6 attr.",
    auc_1=auc_1,
    atributos_2="29 attr.",
    auc_2=auc_2,
    output_path="output",
    campaña="General"
)
